<a href="https://colab.research.google.com/github/jayinc/LLMs4Production/blob/main/LLM_Maven_v2_module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

In [ ]:
# code courtesy of https://nlpforhackers.io/language-models/
import nltk
from nltk.corpus import reuters
nltk.download('reuters')
nltk.download('punkt')
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1

# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dict(model["the","price"])

{'yesterday': 0.004651162790697674,
 'of': 0.3209302325581395,
 'it': 0.05581395348837209,
 'effect': 0.004651162790697674,
 'cut': 0.009302325581395349,
 'for': 0.05116279069767442,
 'paid': 0.013953488372093023,
 'to': 0.05581395348837209,
 'increases': 0.013953488372093023,
 'used': 0.004651162790697674,
 'climate': 0.004651162790697674,
 '.': 0.023255813953488372,
 'cuts': 0.009302325581395349,
 'reductions': 0.004651162790697674,
 'limit': 0.004651162790697674,
 'now': 0.004651162790697674,
 'moved': 0.004651162790697674,
 'per': 0.013953488372093023,
 'adjustments': 0.004651162790697674,
 '(': 0.009302325581395349,
 'slumped': 0.004651162790697674,
 'is': 0.018604651162790697,
 'move': 0.004651162790697674,
 'evolution': 0.004651162790697674,
 'differentials': 0.009302325581395349,
 'went': 0.004651162790697674,
 'the': 0.013953488372093023,
 'factor': 0.004651162790697674,
 'Royal': 0.004651162790697674,
 ',': 0.018604651162790697,
 'again': 0.004651162790697674,
 'changes': 0.0

In [ ]:
import random

# starting words
text = ["today", "the"]
sentence_finished = False

while not sentence_finished:
  # select a random probability threshold
  r = random.random()
  accumulator = .0

  for word in model[tuple(text[-2:])].keys():
      accumulator += model[tuple(text[-2:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  if text[-2:] == [None, None]:
      sentence_finished = True

print (' '.join([t for t in text if t]))

today the Turkish research ship escorted by warships , to boost the incentive .


In [ ]:
!pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 12.3 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "What is the fastest car in the"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next sub-word
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

# Print the predicted word
print(predicted_text)

100%|██████████| 548118077/548118077 [00:16<00:00, 32717032.35B/s]


 What is the fastest car in the world


#Assignment -- Extend the code above to a sentence